# TensorFlow

*TensorFlow* is a powerful open source software library for numerical computation, particularly well suited and fine-tuned for large-scale Machine Learning. Its basic principle is simple: you first define in Python a graph of computations to perform, and then TensorFlow takes that graph and runs it efficiently using optimized C++ code. Most importantly, it is possible to break up the graph into several chunks and run them in parallel across multiple CPUs or GPUs.

In [3]:
import tensorflow as tf

C:\Users\hheim\Anaconda3\envs\TensorFlow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
!conda list

# packages in environment at C:\Users\hheim\Anaconda3\envs\TensorFlow:
#
# Name                    Version                   Build  Channel
_tflow_select             2.1.0                       gpu    anaconda
absl-py                   0.9.0                    py36_0  
argon2-cffi               20.1.0                   pypi_0    pypi
astor                     0.8.1                    py36_0  
attrs                     19.3.0                   pypi_0    pypi
backcall                  0.2.0                      py_0  
blas                      1.0                         mkl  
bleach                    3.1.5                    pypi_0    pypi
ca-certificates           2020.6.24                     0  
certifi                   2020.6.20                py36_0  
cffi                      1.14.2                   pypi_0    pypi
colorama                  0.4.3                      py_0  
cudatoolkit               9.0                           1    anaconda
decorator                 4.4.2     

h5py                      2.7.1            py36he54a1c3_0    anaconda
hdf5                      1.10.1           vc14hb361328_0  [vc14]  anaconda
icc_rt                    2019.0.0             h0cc432a_1    anaconda
importlib-metadata        1.7.0                    py36_0  
intel-openmp              2020.1                      216  
ipykernel                 5.3.4            py36h5ca1d4c_0  
ipython                   7.16.1           py36h5ca1d4c_0  
ipython_genutils          0.2.0                    py36_0  
ipywidgets                7.5.1                    pypi_0    pypi
jedi                      0.17.2                   py36_0  
jinja2                    2.11.2                   pypi_0    pypi
jsonschema                3.2.0                    pypi_0    pypi
jupyter                   1.0.0                    pypi_0    pypi
jupyter-console           6.1.0                    pypi_0    pypi
jupyter_client            6.1.6                      py_0  
jupyter_core              4.6.3   

In [7]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

The most important thing to understand is that this code does not actually perform any computation, even though it looks like it does (especially the last line). It just creates a computation graph. In fact, even the variables are not initialized yet. To evaluate this graph, you need to open a TensorFlow session and use it to initialize the variables and evaluate.

In [8]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42
